In [1]:
#!pip install instaloader

In [2]:
import instaloader
from datetime import datetime
from itertools import dropwhile, takewhile
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from string import punctuation

from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('punkt')

from nltk.stem import PorterStemmer
porter = PorterStemmer()

from nltk.probability import FreqDist

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zhounanlin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zhounanlin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
L = instaloader.Instaloader()

In [4]:
# not using now due to failure to login

with open('../../ins_login_info.txt', encoding='utf-8') as file:
    content = file.read()
    username = content.split()[0]
    pswd = content.split()[1]

# L.login(username, pswd)
# L.save_session_to_file('session_file')

Default value (read from 'ins_account_usernames.txt') for account_usernames is a list of:
["NBCNews", 
 "FoxNews", 
 "CNN", 
 "NYTimes",
 "WashTimes", 
 "WSJ", 
 "BBCNews", 
 "Reuters",
 "Time", 
 "Aljazeeraenglish", 
 "Tabletmag"]

In [5]:
with open('ins_account_usernames.txt', encoding='utf-8') as file:
    content = file.read()
    ins_usernames_list = content.split()

In [6]:
# ins_usernames_list = ins_usernames_list[10:]
# ins_usernames_list

ins_usernames_list = ['NYTimes', 'WSJ', 'Reuters']

For details of properties of Post class:\
https://instaloader.github.io/module/structures.html#posts

For details of properties of Profile class:\
https://instaloader.github.io/module/structures.html?highlight=postcomment#profiles


In [7]:
# for username in account_usernames:
#     df_name = "df_" + username
#     exec_string = "{}.to_csv('latest_ins_csv/{}.csv', index=False)"
#     exec(exec_string.format(df_name, df_name))

**NOTE for this following cell**:\
If successfully scraped for all usernames (no query error popped up), save the exported csv files into a new folder.\
If failed, just skip this cell and run the next cell.

In [8]:
# lists to generate summary info of basic account info
userid_list = []
followers_list = []
full_name_list = []
is_verified_list = []
post_count_list = []    # counts of posts since 231001

for username in ins_usernames_list:
    posts = instaloader.Profile.from_username(L.context, username).get_posts()

    SINCE = datetime(2023, 11, 1)
    UNTIL = datetime.today()
    
    url_code_list = []
    time_utc_list = []
    content_types_list = []
    captions_list = []
    like_counts_list = []
    comment_counts_list = []

    for post in takewhile(lambda p: p.date > SINCE, dropwhile(lambda p: p.date > UNTIL, posts)):
    
        # details of post: date&time, count of likes, count of comments, url
        url_code_list.append(post.shortcode)
        time_utc_list.append(post.date_utc)
        content_types_list.append(post.typename)
        captions_list.append(post.caption)
        like_counts_list.append(post.likes)
        comment_counts_list.append(post.comments)
        # L.download_post(post, username)
    
    # create df with info of each posts from this account
    df_name = "df_" + username
    exec_string = """{} = pd.DataFrame(data={{
    'owner_username': [username] * len(url_code_list),
    'url_code': url_code_list, 
    'time_utc': pd.to_datetime(time_utc_list), 
    'type': content_types_list,
    'caption': captions_list,
    'likes': like_counts_list, 
    'comments': comment_counts_list}})
    """
    exec(exec_string.format(df_name))
    
    # export df to csv
    exec_string = "{}.to_csv('latest_ins_csv/{}.csv', index=False)"
    exec(exec_string.format(df_name, df_name))
    
    print("Done with posts of", username)
    print("{} has {} posts since Nov 1st.".format(username, len(url_code_list)))
    post_count_list.append(len(url_code_list))
    
    
    # append this account info
    userid_list.append(post.owner_profile.userid)
    followers_list.append(post.owner_profile.followers)
    full_name_list.append(post.owner_profile.full_name)
    is_verified_list.append(post.owner_profile.is_verified)
    
d = {'user_id': userid_list, 'followers': followers_list, 'full_name':full_name_list, 'is_verified':is_verified_list, 'posts_since_231001': post_count_list}
df_ins_accounts = pd.DataFrame(data=d)
df_ins_accounts.to_csv('latest_ins_csv/df_ins_accounts.csv', index=False)

Done with posts of NYTimes
NYTimes has 181 posts since Nov 1st.
Done with posts of WSJ
WSJ has 66 posts since Nov 1st.
Done with posts of Reuters
Reuters has 41 posts since Nov 1st.


In [9]:
stacked_dfs = pd.DataFrame()

try:
    if len(df_ins_accounts) == len(ins_usernames_list):
        print("Successfully scraped for all usernames.")
        print("Remember to save the exported csv files in the directory of './latest_ins_csv' into a new folder")
        
        for username in ins_usernames_list:
            df_name = "df_" + username
            exec("stacked_dfs=pd.concat([stacked_dfs, {}], ignore_index=True)".format(df_name))
            
    else:
        print("Not all usernames were successfully scraped.")
        print("Use backup files instead.")
        
        df_ins_accounts = pd.read_csv('backup_ins_df_231101/df_ins_accounts.csv')
        for username in ins_usernames_list:
            df_name = "df_" + username
            path = 'backup_ins_df_231101/' + df_name + ".csv"
            exec("stacked_dfs=pd.concat([stacked_dfs, pd.read_csv('{}')], ignore_index=True)".format(path))

except:
    print("Failed to scrape.")
    print("Use backup files instead.")
    
    df_ins_accounts = pd.read_csv('backup_ins_df_231101/df_ins_accounts.csv')
    for username in ins_usernames_list:
        df_name = "df_" + username
        path = 'backup_ins_df_231101/' + df_name + ".csv"
        exec("stacked_dfs=pd.concat([stacked_dfs, pd.read_csv('{}')], ignore_index=True)".format(path))

Successfully scraped for all usernames.
Remember to save the exported csv files in the directory of './latest_ins_csv' into a new folder


In [10]:
# from HW2
def find_freq_tokens(tokens, display_limit):
    
    freq_dict = FreqDist(tokens)
    bigram_freq = dict()
    
    sorted_items = sorted(freq_dict.items(), key=lambda item: item[1], reverse=True)
    top_keys = [item[0] for item in sorted_items[:display_limit]]
    
    return top_keys

In [11]:
def text_to_tokens(text):
    
    lower_str = text.lower()
    no_punc_str = ''.join([character for character in lower_str if character not in punctuation])
    no_digit_str = ''.join([character for character in no_punc_str if not character.isdigit()])
    no_special_charac = ''.join([character for character in no_digit_str if character not in ['\u2060', '”', '“']])
    no_dash = no_special_charac.replace('—', ' ').replace('-', ' ').replace('’', ' ')
    orig_tokens = word_tokenize(no_dash)
    
    tokens = [porter.stem(word) for word in orig_tokens if not word in stop_words]
    
    return tokens

In [12]:
stacked_dfs = stacked_dfs.dropna(subset=['caption'])

In [13]:
all_ins_tokens = []

for index, row in stacked_dfs.iterrows():

    tokens = text_to_tokens(row['caption'])
    all_ins_tokens += tokens

top_tokens = find_freq_tokens(all_ins_tokens, 30)
top_tokens

['link',
 'bio',
 'read',
 'new',
 'photo',
 'said',
 'year',
 'gaza',
 'peopl',
 'israel',
 'time',
 'tap',
 'one',
 'york',
 'citi',
 'us',
 'isra',
 'say',
 'hama',
 'hospit',
 'war',
 'like',
 'famili',
 'palestinian',
 'first',
 'state',
 'includ',
 'accord',
 'last',
 'mani']

## Pick out the posts that talks about gaza conflict
- method 1: find the posts that contain at least one of the tokens in gaza_tokens_1
- method 2: find the posts that contain at least one of the tokens in gaza_tokens_2, which has two more tokens: "war" and "attack"
- method 3: use a language model to pick

**method 1**

In [14]:
gaza_tokens_1 = ['israel',
 'gaza',
 'isra',
 'hama',
 'palestinian',
 'palestin',
 'israelgazawar',
 'israelhama',
 'israelgaza']


is_about_gaza_1 = []

for index, row in stacked_dfs.iterrows():

    tokens = text_to_tokens(row['caption'])

    found = False
    for g_token in gaza_tokens_1:
        if g_token in tokens:
            found = True
            break

    is_about_gaza_1.append(found)

stacked_dfs['is_about_gaza_1'] = is_about_gaza_1

stacked_dfs[stacked_dfs['is_about_gaza_1']==True]

,owner_username,url_code,time_utc,type,caption,likes,comments,is_about_gaza_1
0,NYTimes,C0HtVJPOpEd,2023-11-26 19:10:47,GraphVideo,An American Girl Among Hostages Released on Su...,11321,3238,True
6,NYTimes,C0FGUCauzjN,2023-11-25 18:50:41,GraphVideo,Hostages Reunite With Families at Israeli Chil...,78504,5067,True
7,NYTimes,C0EzyNaOUe3,2023-11-25 16:07:26,GraphSidecar,"Moderated by Emily Bazelon, a staff writer for...",19828,2222,True
13,NYTimes,C0CKllUOf4o,2023-11-24 15:28:58,GraphImage,"Some two dozen hostages held in Gaza, includin...",19439,2159,True
24,NYTimes,Cz9QIwMLoiG,2023-11-22 17:41:51,GraphVideo,As Israel and Hamas Move Toward Hostages’ Rele...,12130,3941,True
...,...,...,...,...,...,...,...,...
282,Reuters,CzRKeiSLjZF,2023-11-05 14:45:16,GraphImage,Palestinians gather at the site of an Israeli ...,3352,504,True
283,Reuters,CzMXMF5xmf4,2023-11-03 18:00:05,GraphSidecar,Medics trapped in a clinic tending to the woun...,8309,2994,True
284,Reuters,CzL7SijL7rp,2023-11-03 13:56:20,GraphImage,Palestinians gather at the site of Israeli str...,3727,815,True
286,Reuters,CzHcfULLqwo,2023-11-01 20:10:15,GraphImage,Israeli soldiers from the IDF's Home Front Com...,5070,1275,True


In [15]:
all_ins_GAZA_tokens = []

for index, row in stacked_dfs[stacked_dfs['is_about_gaza_1']==True].iterrows():

    tokens = text_to_tokens(row['caption'])
    all_ins_GAZA_tokens += tokens

len(all_ins_GAZA_tokens)

5433

In [16]:
all_ins_GAZA_tokens = []

for index, row in stacked_dfs[stacked_dfs['is_about_gaza_1']==True].iterrows():

    tokens = word_tokenize(row['caption'])
    all_ins_GAZA_tokens += tokens

len(all_ins_GAZA_tokens)

10547

**method 2**

In [17]:
gaza_tokens_2 = ['israel',
 'gaza',
 'isra',
 'hama',
 'palestinian',
 'attack',
 'palestin',
 'war',
 'israelgazawar',
 'israelhama',
 'israelgaza']


is_about_gaza_2 = []

for index, row in stacked_dfs.iterrows():

    tokens = text_to_tokens(row['caption'])

    found = False
    for g_token in gaza_tokens_2:
        if g_token in tokens:
            found = True
            break

    is_about_gaza_2.append(found)

stacked_dfs['is_about_gaza_2'] = is_about_gaza_2

stacked_dfs[stacked_dfs['is_about_gaza_2']==True]

,owner_username,url_code,time_utc,type,caption,likes,comments,is_about_gaza_1,is_about_gaza_2
0,NYTimes,C0HtVJPOpEd,2023-11-26 19:10:47,GraphVideo,An American Girl Among Hostages Released on Su...,11321,3238,True,True
6,NYTimes,C0FGUCauzjN,2023-11-25 18:50:41,GraphVideo,Hostages Reunite With Families at Israeli Chil...,78504,5067,True,True
7,NYTimes,C0EzyNaOUe3,2023-11-25 16:07:26,GraphSidecar,"Moderated by Emily Bazelon, a staff writer for...",19828,2222,True,True
13,NYTimes,C0CKllUOf4o,2023-11-24 15:28:58,GraphImage,"Some two dozen hostages held in Gaza, includin...",19439,2159,True,True
24,NYTimes,Cz9QIwMLoiG,2023-11-22 17:41:51,GraphVideo,As Israel and Hamas Move Toward Hostages’ Rele...,12130,3941,True,True
...,...,...,...,...,...,...,...,...,...
282,Reuters,CzRKeiSLjZF,2023-11-05 14:45:16,GraphImage,Palestinians gather at the site of an Israeli ...,3352,504,True,True
283,Reuters,CzMXMF5xmf4,2023-11-03 18:00:05,GraphSidecar,Medics trapped in a clinic tending to the woun...,8309,2994,True,True
284,Reuters,CzL7SijL7rp,2023-11-03 13:56:20,GraphImage,Palestinians gather at the site of Israeli str...,3727,815,True,True
286,Reuters,CzHcfULLqwo,2023-11-01 20:10:15,GraphImage,Israeli soldiers from the IDF's Home Front Com...,5070,1275,True,True


In [18]:
df_diff = stacked_dfs[stacked_dfs['is_about_gaza_1'] != stacked_dfs['is_about_gaza_2']]
df_diff

,owner_username,url_code,time_utc,type,caption,likes,comments,is_about_gaza_1,is_about_gaza_2
46,NYTimes,Czza3_5LGs0,2023-11-18 22:02:40,GraphSidecar,"Half a century ago, caimans, aquatic reptiles,...",11419,102,False,True
75,NYTimes,Czo-ArJP2tY,2023-11-14 20:39:00,GraphImage,Doctors have long relied on a few key patient ...,11931,210,False,True
89,NYTimes,Czj2lk8uyvr,2023-11-12 20:59:51,GraphVideo,Army Ammunition Plant Is Tied to Mass Shooting...,11861,953,False,True
99,NYTimes,Cze-kcpM9Cx,2023-11-11 00:00:35,GraphImage,Barry Manilow is finally getting his wish: a B...,14766,215,False,True
130,NYTimes,CzWSu44ONIZ,2023-11-07 14:35:32,GraphImage,"For more than a century, Lessie Benningfield R...",202030,977,False,True
134,NYTimes,CzUaX-aAD9c,2023-11-06 21:01:51,GraphSidecar,When @rebeccayarros pitched her publisher a se...,37523,342,False,True
166,NYTimes,CzJvOsFANaQ,2023-11-02 17:32:26,GraphSidecar,"On a Saturday in October, in a small foundry, ...",14188,502,False,True
169,NYTimes,CzJNzn3rneB,2023-11-02 12:40:23,GraphSidecar,"Artem Dymyd, Dmytro Paschuk and Roman Lozynsky...",41107,1303,False,True
189,WSJ,Cz9LssiS_Y9,2023-11-22 17:03:34,GraphImage,Teen boys are falling for an online nude-photo...,11712,310,False,True
251,Reuters,Cz9xXQzOi7v,2023-11-22 22:32:20,GraphVideo,A vehicle exploded on the Rainbow Bridge linki...,1586,227,False,True


In [19]:
len(df_diff)

10

In [20]:
for index, row in df_diff.iterrows():

    print(row['caption'])
    print("------------")

Half a century ago, caimans, aquatic reptiles, were taken from their homelands in the tropical wetlands of Latin America and brought to the U.S. for the leather and pet trades. Most of the hundreds of thousands of caimans imported into the country during the 1970s perished, but a few held out in the southernmost corner of Florida. Like many invasive species there, they’ve become a nuisance. And they’re more aggressive when cornered than native crocodilians, potentially putting people at risk of attacks.

Tap the link in our bio to learn more about the elusive caimans, and the effort to get them under control. Photos by @jason_gulley_science
------------
Doctors have long relied on a few key patient characteristics to assess risk of a heart attack or stroke, using a calculus that considers blood pressure, cholesterol, smoking and diabetes status, as well as demographics: age, sex and race.

Now, the American Heart Association is taking race out of the equation.

The overhaul of the wide

**Method 3**

## Export the stacked dataframe to csv

In [21]:
stacked_dfs.to_csv('stacked_ins_df.csv', index=False)

Skipping hashtag due to lack of useful posts from the limited and ramdomly fetched posts.

In [22]:
# posts = instaloader.Instaloader().get_hashtag_posts("gaza")
# posts = instaloader.Hashtag.from_name(L.context, "gaza").get_top_posts()

Only top 9 posts of a hashtag can be fetched with the following method

In [23]:
# posts = instaloader.Hashtag.from_name(L.context, "gaza").get_top_posts()